In [13]:
import tensorflow as tf
import numpy as np
import os
#import MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [14]:
sess = tf.InteractiveSession() #initialize session

#input placeholders
with tf.name_scope('x'):
    x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [15]:
#function definitions
def weight_variable(shape, name):
  initial = tf.truncated_normal(shape, stddev=0.1, name=name)
  return tf.Variable(initial)

def bias_variable(shape, name):
  initial = tf.constant(0.1, shape=shape, name=name)
  return tf.Variable(initial)

def conv2d(x, W, name):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name=name)

def max_pool_2x2(x,name):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME', name=name)

In [16]:
W_conv1 = weight_variable([5, 5, 1, 32], name='W_C1')
b_conv1 = bias_variable([32], name='B_C1')

x_image = tf.reshape(x, [-1,28,28,1]) #vectorize the image

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1 , name='conv_1') + b_conv1)
h_pool1 = max_pool_2x2(h_conv1, name='pool_1')

W_conv2 = weight_variable([5, 5, 32, 64], name='W_C2')
b_conv2 = bias_variable([64], name='B_C2')

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, name='conv_2') + b_conv2)
h_pool2 = max_pool_2x2(h_conv2, name='pool_2')

W_fc1 = weight_variable([7 * 7 * 64, 1024], name='W_FC1')
b_fc1 = bias_variable([1024], name='B_FC1')

feature_vector = tf.reshape(h_pool2, [-1, 7*7*64])

h_fc1 = tf.nn.relu(tf.matmul(feature_vector, W_fc1) + b_fc1, name='FC_1')

W_fc2 = weight_variable([1024, 10], name='W_FC2')
b_fc2 = bias_variable([10], name='B_FC2')

with tf.name_scope('logits'):
    logits = tf.add(tf.matmul(h_fc1, W_fc2), b_fc2, name='logits')

y = tf.nn.softmax(logits, name='softmax_prediction')

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

# we only need these two to make inference using the trained model
tf.add_to_collection("logits", logits)
tf.add_to_collection("x", x)

In [17]:
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())

In [ ]:
for i in range(500):
    batch = mnist.train.next_batch(100)
    if i%100 == 0:
        train_acc = accuracy.eval(feed_dict={
            x:batch[0], y_:batch[1]
        })
        print("Step %d, training accuracy %g"%(i, train_acc))
    train_step.run(feed_dict={x:batch[0], y_:batch[1]})

current_dir = os.getcwd()
saver.save(sess, current_dir + '/model/mnist.ckpt')
print("Training is finished.")

Step 0, training accuracy 0.08
